## Import Packages

In [1]:
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl
import sqlite3
import numpy as np

## Crawlering & Storage

In [2]:
"""Create database"""
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except :
        print("Connection error")
    return conn

def create_table(conn, create_table):
    try:
        c = conn.cursor()
        c.execute(create_table)
        #c.commit()
    except :
        print("Create table error")

#def main():
    database = "mydatabase1.db"
    # create a database connection
    conn = create_connection(database)

    sql_create_ptt_table_query = """ CREATE TABLE IF NOT EXISTS PTT (
                                        board text NOT NULL,
                                        id text,
                                        pushes text,
                                        author text,
                                        title text
                                    ); """
    
    # create tables
    if conn is not None:
        # create ptt table
        create_table(conn, sql_create_ptt_table_query)
    else:
        print("Error! cannot create the database connection.")

    conn.close()

main()


In [3]:
'''New Data'''
def add_new_row(insertlist):
    try:
        conn = sqlite3.connect('mydatabase1.db')
        cursor = conn.cursor()
        strinsertlist = "'"+"','".join(insertlist)+"'"
        #print(strinsertlist)
        add_new_row_query = """INSERT INTO PTT(board,id,pushes,author,title) VALUES("""+strinsertlist+""") """
        
        cursor.execute(add_new_row_query)
        conn.commit()
        #print("ok",insertlist)
        conn.close()
    except:
        print('?')
        print(insertlist)
        conn.close()



In [ ]:
"""用openpyxl寫進worksheet"""
wb = openpyxl.Workbook()
ws = wb.active
ws.append(["看板名稱",'id',"推文數","文章作者","文章名稱"]) 
#for board_name in ('NBA','Baseball'):
for board_name in ('NBA',"Stock",'Boy-Girl','Baseball',"Lifeismoney"):
    url = 'https://www.ptt.cc/bbs/'+board_name+'/index.html'
    for page in range(1,31): #30頁
        print(url)  #當前的url
        print(board_name,"版page",page)
        time.sleep(2)
        web = requests.get(url,cookies={'over18':'1'})
        soup = BeautifulSoup(web.text, "html.parser")
        pushes = soup.find_all('div',class_='nrec')
        authors = soup.find_all('div',class_='author')
        titles = soup.find_all('div', class_='title')     # 取得 class 為 title 的 div 內容
        for i,j,k in zip(titles,pushes,authors):
            if i.find('a') != None:                         # 判斷如果不為 None
                a = j.text
                #print(a,k.text,end=" ")        #推文數,作者
                #print(i.find('a').get_text())                 # 取得 div 裡 a 的內容，使用 get_text() 取得文字
                
                id = i.find('a').get("href")[i.find('a').get("href").find(board_name):i.find('a').get("href").find('.A.')]
                
                b=0
                
                if a =='爆':
                    url_article = "https://www.ptt.cc"+i.find('a').get('href')
                    web_article = requests.get(url_article,cookies={'over18':'1'})
                    soup_article = BeautifulSoup(web_article.text, "html.parser")
                    b=0
                    pushnumber = soup_article.find_all('div',class_='push')
                    for m in pushnumber:
                        if m.text[0] == '推':
                            b+=1
                        elif m.text[0] == '噓':
                            b-=1
                    #print("爆",b)
                #print([board_name,a,k.text,i.text])
                
                if b>0:
                    #print([board_name,str(b),k.text,i.text])
                    data = [board_name,id,str(b),k.text,i.text]
                    ws.append(data)
                    add_new_row(data)
                else:
                    #print([board_name,a,k.text,i.text])
                    data = [board_name,id,a,k.text,i.text]
                    ws.append(data)
                    add_new_row(data)

                

        url = "https://www.ptt.cc"+soup.find_all("a",class_='btn wide')[1].get('href') #上一頁的href

wb.save("PPT1.xlsx")
